In [1]:
import os
os.chdir('E:\\test')
import pandas as pd
import numpy as np
import networkx as nx
import sys
import time
import matplotlib.pyplot as plt
def find_community(graph,k):
    return list(nx.k_clique_communities(graph,k))

In [2]:
source_target = pd.read_csv('source_target.csv',encoding='utf-8',low_memory=False)
source_target = source_target.dropna(axis=0).reset_index()
#source_target.head()

In [3]:
source = source_target['source']
target = source_target['target']
count = source_target['count']
n = source.count()
G = nx.DiGraph()

for i in range(n):
    G.add_edge(source[i],target[i],weight = count[i])

G.remove_edges_from(G.selfloop_edges())
G.number_of_nodes()

635611

function:求出中心度，入度，出度，紧密中心度

In [4]:
def degree_in_out(G):
    a = G.degree()
    arr = list(a.items())
    a1 = pd.DataFrame(arr)
    a1.columns = ['node','degree']

    b = G.in_degree()
    brr = list(b.items())
    b1 = pd.DataFrame(brr)
    b1.columns = ['node','in_degree']

    c = G.out_degree()
    crr = list(c.items())
    c1 = pd.DataFrame(crr)
    c1.columns = ['node','out_degree']
    
    #closeness = pd.DataFrame(list(nx.closeness_centrality(G).items()),columns=['node','closeness_centrality'])
    #betweeness = pd.DataFrame(list(nx.betweenness_centrality(G)),columns=['node','betweenness_centrality'])
    
    e = pd.merge(a1,b1,how='left',on='node')
    f = pd.merge(e,c1,how='left',on='node')
    #g = pd.merge(f,closeness,how = 'left',on='node')
    #h = pd.merge(g,betweeness,how='left',on='node')
    return f

In [5]:
degree_df = degree_in_out(G)

In [6]:
out1 = degree_df[degree_df['out_degree']>1]
G_out1 = G.subgraph(out1['node'])

画度数对数坐标图

In [7]:
# ###### import matplotlib.pyplot as plt  #导入科学绘图的matplotlib包
# %matplotlib inline   
# hist =  nx.degree_histogram(G)          #返回图中所有节点的度分布序列
# x = range(len(hist))                             #生成x轴序列，从1到最大度
# y = [z / float(sum(hist)) for z in hist]  
# #将频次转换为频率，这用到Python的一个小技巧：列表内涵，Python的确很方便：）
# plt.loglog(x,y,color="blue",linewidth=1)           #在双对数坐标轴上绘制度分布曲线  
# plt.show()                                                          #显示图表

注意：下面使用H，H为无向图。nx.k_clique_communities不能使用有向图

In [8]:
#degree_df.describe()

In [9]:
H = nx.Graph()
for i in range(n):
    H.add_edge(source[i],target[i])
H.remove_edges_from(H.selfloop_edges())
out1 = degree_df[degree_df['out_degree']>1]#出度大于1
H_out1 = H.subgraph(out1['node'])
in3_high = degree_df[degree_df['in_degree']>=3]#入度大于等于3
H_in3_high = H.subgraph(in3_high['node'])
in3_less = degree_df[degree_df['in_degree']<3]#入度小于3
H_in3_less = H.subgraph(in3_less['node'])
# for k in range(2,5):
#     print('输入k：',k)
#     start_time = time.clock()
#     rst_com = find_community(H,k)
#     end_time = time.clock()
#     print('计算用时：',(end_time-start_time))
#     print('聚类个数：',len(rst_com))
#     print(rst_com)

In [10]:
#发现社区
#1.出度大于1
out1_com = find_community(H_out1,5)
df_out1_com = pd.DataFrame(out1_com)
#2.入度大于3
in3_high_com = find_community(H_in3_high,5)
df_in3_high_com = pd.DataFrame(in3_high_com)
#3.入度小于3
in3_less_com = find_community(H_in3_less,3)
df_in3_less_com = pd.DataFrame(in3_less_com)
#4.全部数据
H_com = find_community(H,3)
df_H_com = pd.DataFrame(H_com)

In [11]:
# K-core
# graphs = nx.connected_component_subgraphs(nx.k_core(G_out1,k=3))
# for g in graphs:
#     print(g.edges())
graphs = nx.k_core(G_out1,3)
a =pd.DataFrame(graphs.edges(),columns=['Source','Target'])
#?nx.k_nearest_neighbors(G=G_out1)

读取黑名单

In [30]:
blacknodes = pd.read_csv('e:/blacknodes.csv',encoding='gb2312',low_memory=False)
blacknodes = blacknodes.dropna()
blacknodes = blacknodes[blacknodes['black']!='0']
del blacknodes['Unnamed: 0']
blacknodes = blacknodes.reset_index()
del blacknodes['index']
#blacknodes.head()

In [20]:
df_out1_com['black']=0
for z in range(len(df_out1_com)):
    for i in range(len(df_out1_com.ix[z])):
        for j in range(len(blacknodes)):
            if df_out1_com.ix[z,i]==blacknodes.ix[j,'Id']:
                df_out1_com.ix[z,'black']='黑名单'

In [44]:
#注意，这是当k-clique参数设置为3时，筛选人数在20个以内的群体
out1_com = find_community(H_out1,3)
df_out1_com = pd.DataFrame(out1_com)
df_out1_com['black']=0
for z in range(len(df_out1_com)):
    for i in range(len(df_out1_com.ix[z])):
        for j in range(len(blacknodes)):
            if df_out1_com.ix[z,i]==blacknodes.ix[j,'Id']:
                df_out1_com.ix[z,'black']='黑名单'
df_out1_com['count_ph'] = df_out1_com.apply(lambda e: e.count()-1,axis=1)
a = df_out1_com[df_out1_com['count_ph']<=20]
a = a[a.columns[0:20]]
a['black'] = df_out1_com.ix[a.index]['black']
a.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,black
0,17602397034,13594043754,17623005473,15123976881,18580725240,15736225316,13272742820,None,None,None,...,None,None,None,None,None,None,None,None,None,0
2,13117716206,18677170204,18260936233,13277884426,17677139916,18376640456,15607808953,18249993610,17776685736,15778309218,...,None,None,None,None,None,None,None,None,None,0
4,17782005692,051288178230,18223044644,15696895998,13028361226,15730183286,052380554888,02395188,15084438321,13368380673,...,15025357076,15736045473,051288178227,18581141941,18581061020,None,None,None,None,0
5,18213079089,15187751749,18213445422,13529242084,13529364095,18313372665,13529226814,15125534718,None,None,...,None,None,None,None,None,None,None,None,None,0
6,17878699494,15777601142,15078194715,18377612703,15878494241,17878699824,13677768346,None,None,None,...,None,None,None,None,None,None,None,None,None,0


In [19]:
df_in3_high_com['black']=0
for z in range(len(df_in3_high_com)):
    for i in range(len(df_in3_high_com.ix[z])): 
        for j in range(len(blacknodes)):
            if df_in3_high_com.ix[z][i]==blacknodes.ix[j,'Id'] and blacknodes.ix[j,'black']=='黑名单':
                df_in3_high_com.ix[z,'black']='黑名单'
            elif df_in3_high_com.ix[z][i]==blacknodes.ix[j,'Id'] and blacknodes.ix[j,'black']=='灰名单':
                df_in3_high_com.ix[z,'black']='灰名单'
        if df_in3_high_com.ix[z,'black']=='黑名单':
            break

In [16]:
df_in3_less_com['black']='0'
for z in range(len(df_in3_less_com)):
    for i in range(len(df_in3_less_com.ix[z])): 
        if df_in3_less_com.ix[z,'black']=='黑名单':
            break
        for j in range(len(blacknodes)):
            if df_out1_com.ix[z,i]==blacknodes.ix[j,'Id'] and blacknodes.ix[j,'black']=='黑名单':
                df_in3_less_com.ix[z,'black']='黑名单'
            elif df_out1_com.ix[z,i]==blacknodes.ix[j,'Id'] and blacknodes.ix[j,'black']=='灰名单':
                df_in3_less_com.ix[z,'black']='灰名单'

In [18]:
# 全部数据直接做网络分析
# df_H_com['black']='0'
# for z in range(len(df_H_com)):
#     for i in range(len(df_H_com.ix[z])): 
#         if df_H_com.ix[z,'black']=='黑名单':
#             break
#         for j in range(len(blacknodes)):
#             if df_H_com.ix[z][i]==blacknodes.ix[j,'Id'] and blacknodes.ix[j,'black']=='黑名单':
#                 df_H_com.ix[z,'black']='黑名单'
#             elif df_H_com.ix[z][i]==blacknodes.ix[j,'Id'] and blacknodes.ix[j,'black']=='灰名单':
#                 df_H_com.ix[z,'black']='灰名单'

In [18]:
df_H_com.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,black
0,18227584400,15877875476,15587200782,18227598856,15023228294,18783523772,18387013239,15087182842,None,None,None,None,None,None,None,None,None,None,None,0
1,18523588360,13042323298,15917751013,17723244304,15728682661,15523094777,15023790825,18223625787,13042323286,17623196710,13638365247,17783002185,15523223888,18875023150,17782315712,15683966818,15558863886,15683195888,15523881816,0
2,15778309218,18376640456,18249993610,18260936233,18677170204,13117716206,None,None,None,None,None,None,None,None,None,None,None,None,None,黑名单
3,18186105866,15071256936,18672330956,18611898613,18683620420,15221116978,None,None,None,None,None,None,None,None,None,None,None,None,None,黑名单


In [ ]:
df_in3_less_com.to_csv('df_in3_less_com.csv',index=False)
df_out1_com.to_csv('df_out1_com.csv',index=False)
df_in3_high_com.to_csv('df_in3_high_com.csv',index=False)